In [1]:
from pyod.models.ecod import ECOD  # ECOD detector
from pyod.models.knn import KNN   # kNN detector
from joblib import dump, load
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from loader import Loader
import os

In [2]:
clf_name = "KNN"
# clf = ECOD()
clf = KNN()


## Data Loading and Pre-Processing

In [3]:
loader = Loader()
if not os.path.exists("data/data.pkl"):
    loader.extract()
    df = loader.create_df_from_xlsx()
    df = loader.process_df(df)
    loader.save_df(df)
else:
    df = loader.load_df()


In [4]:
print(df.shape)
print(df.columns)
print(df.info())

(15232, 19)
Index(['کد محور', 'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'تعداد وسیله نقلیه سبک', 'تعداد وسیله نقلیه سنگین', 'سرعت متوسط',
       'تعداد تخلف سرعت غیر مجاز', 'تعداد تخلف فاصله غیر مجاز',
       'تعداد تخلف سبقت غیر مجاز', 'تعداد وسیله نقلیه کلاس ۱',
       'تعداد وسیله نقلیه کلاس ۲', 'تعداد وسیله نقلیه کلاس ۳',
       'تعداد وسیله نقلیه کلاس ۴', 'تعداد وسیله نقلیه کلاس ۵',
       'تعداد برآورد شده', 'holiday', 'date', 'day_of_week', 'time_start'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15232 entries, 0 to 30
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    15232 non-null  int64  
 1   مدت زمان کارکرد (دقیقه)    15232 non-null  int64  
 2   تعداد کل وسیله نقلیه       15232 non-null  int64  
 3   تعداد وسیله نقلیه سبک      500 non-null    float64
 4   تعداد وسیله نقلیه سنگین    500 non-null    float

In [5]:
df.drop(columns=['تعداد وسیله نقلیه سنگین','تعداد وسیله نقلیه سبک','تعداد برآورد شده']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15232 entries, 0 to 30
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    15232 non-null  int64  
 1   مدت زمان کارکرد (دقیقه)    15232 non-null  int64  
 2   تعداد کل وسیله نقلیه       15232 non-null  int64  
 3   سرعت متوسط                 15232 non-null  float64
 4   تعداد تخلف سرعت غیر مجاز   15232 non-null  int64  
 5   تعداد تخلف فاصله غیر مجاز  15232 non-null  int64  
 6   تعداد تخلف سبقت غیر مجاز   15232 non-null  int64  
 7   تعداد وسیله نقلیه کلاس ۱   14732 non-null  float64
 8   تعداد وسیله نقلیه کلاس ۲   14732 non-null  float64
 9   تعداد وسیله نقلیه کلاس ۳   14732 non-null  float64
 10  تعداد وسیله نقلیه کلاس ۴   14732 non-null  float64
 11  تعداد وسیله نقلیه کلاس ۵   14732 non-null  float64
 12  holiday                    15232 non-null  object 
 13  date                       15232 non-null  object

In [6]:
df = df.drop(columns=['تعداد وسیله نقلیه سنگین','تعداد وسیله نقلیه سبک','تعداد برآورد شده'])

In [8]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14732 entries, 0 to 30
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    14732 non-null  int64  
 1   مدت زمان کارکرد (دقیقه)    14732 non-null  int64  
 2   تعداد کل وسیله نقلیه       14732 non-null  int64  
 3   سرعت متوسط                 14732 non-null  float64
 4   تعداد تخلف سرعت غیر مجاز   14732 non-null  int64  
 5   تعداد تخلف فاصله غیر مجاز  14732 non-null  int64  
 6   تعداد تخلف سبقت غیر مجاز   14732 non-null  int64  
 7   تعداد وسیله نقلیه کلاس ۱   14732 non-null  float64
 8   تعداد وسیله نقلیه کلاس ۲   14732 non-null  float64
 9   تعداد وسیله نقلیه کلاس ۳   14732 non-null  float64
 10  تعداد وسیله نقلیه کلاس ۴   14732 non-null  float64
 11  تعداد وسیله نقلیه کلاس ۵   14732 non-null  float64
 12  holiday                    14732 non-null  object 
 13  date                       14732 non-null  object

In [9]:
df = df.dropna()

In [11]:
df[df["holiday"] == True].head()

,کد محور,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,سرعت متوسط,تعداد تخلف سرعت غیر مجاز,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد وسیله نقلیه کلاس ۱,تعداد وسیله نقلیه کلاس ۲,تعداد وسیله نقلیه کلاس ۳,تعداد وسیله نقلیه کلاس ۴,تعداد وسیله نقلیه کلاس ۵,holiday,date,day_of_week,time_start
0,113401,1440,2160,65.0,22,456,109,2122.0,25.0,5.0,5.0,3.0,True,1395-01-01,1,00:00:00
1,113401,1360,2069,60.0,5,380,96,2014.0,43.0,5.0,2.0,5.0,True,1395-01-02,2,00:00:00
2,113401,1440,2419,63.0,7,475,102,2333.0,80.0,4.0,2.0,0.0,True,1395-01-03,3,00:00:00
3,113401,1440,2624,63.0,2,580,88,2569.0,42.0,6.0,2.0,5.0,True,1395-01-04,4,00:00:00
11,113401,1440,3799,61.0,6,1082,156,3698.0,90.0,5.0,4.0,2.0,True,1395-01-12,5,00:00:00


## Some Plots

In [14]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [18]:
df.sort_values(by=['date'],inplace=True)
df.head()

,کد محور,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,سرعت متوسط,تعداد تخلف سرعت غیر مجاز,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد وسیله نقلیه کلاس ۱,تعداد وسیله نقلیه کلاس ۲,تعداد وسیله نقلیه کلاس ۳,تعداد وسیله نقلیه کلاس ۴,تعداد وسیله نقلیه کلاس ۵,holiday,date,day_of_week,time_start
0,113401,1440,2160,65.00,22,456,109,2122.0,25.0,5.0,5.0,3.0,True,1395-01-01,1,00:00:00
0,113903,1440,24328,85.00,221,4687,0,23693.0,348.0,132.0,56.0,99.0,True,1395-01-01,1,00:00:00
0,114351,1435,8642,106.00,875,826,0,8341.0,193.0,19.0,15.0,74.0,True,1395-01-01,1,00:00:00
0,114206,465,4216,98.00,82,393,0,4155.0,50.0,10.0,1.0,0.0,True,1395-01-01,1,00:00:00
0,113211,1440,309,82.82,119,11,18,222.0,44.0,14.0,2.0,27.0,True,1395-01-01,1,00:00:00


In [19]:
df.tail()

,کد محور,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,سرعت متوسط,تعداد تخلف سرعت غیر مجاز,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد وسیله نقلیه کلاس ۱,تعداد وسیله نقلیه کلاس ۲,تعداد وسیله نقلیه کلاس ۳,تعداد وسیله نقلیه کلاس ۴,تعداد وسیله نقلیه کلاس ۵,holiday,date,day_of_week,time_start
30,114151,1440,16584,69.34,211,2863,0,11582.0,2316.0,636.0,259.0,1791.0,False,1395-01-31,3,00:00:00
30,113263,580,10254,87.00,174,2196,0,10163.0,53.0,28.0,4.0,6.0,False,1395-01-31,3,00:00:00
30,113251,1440,1585,91.63,44,22,47,812.0,146.0,213.0,146.0,268.0,False,1395-01-31,3,00:00:00
30,114152,1370,16646,68.00,3202,933,0,15581.0,647.0,128.0,163.0,127.0,False,1395-01-31,3,00:00:00
30,113264,1440,7188,70.45,40,459,0,7117.0,45.0,21.0,4.0,1.0,False,1395-01-31,3,00:00:00


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14732 entries, 0 to 30
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    14732 non-null  int64  
 1   مدت زمان کارکرد (دقیقه)    14732 non-null  int64  
 2   تعداد کل وسیله نقلیه       14732 non-null  int64  
 3   سرعت متوسط                 14732 non-null  float64
 4   تعداد تخلف سرعت غیر مجاز   14732 non-null  int64  
 5   تعداد تخلف فاصله غیر مجاز  14732 non-null  int64  
 6   تعداد تخلف سبقت غیر مجاز   14732 non-null  int64  
 7   تعداد وسیله نقلیه کلاس ۱   14732 non-null  float64
 8   تعداد وسیله نقلیه کلاس ۲   14732 non-null  float64
 9   تعداد وسیله نقلیه کلاس ۳   14732 non-null  float64
 10  تعداد وسیله نقلیه کلاس ۴   14732 non-null  float64
 11  تعداد وسیله نقلیه کلاس ۵   14732 non-null  float64
 12  holiday                    14732 non-null  object 
 13  date                       14732 non-null  object

In [30]:
df[['date']].value_counts()

date      
1395-01-01    512
1395-01-02    512
1395-01-03    512
1395-01-04    512
1395-01-05    511
1395-01-06    509
1395-01-07    509
1395-01-08    508
1395-01-09    502
1395-01-10    498
1395-01-11    496
1395-01-12    491
1395-01-13    491
1395-01-14    491
1395-01-16    485
1395-01-15    485
1395-01-17    475
1395-01-18    472
1395-01-19    468
1395-01-20    468
1395-01-21    466
1395-01-22    466
1395-01-23    462
1395-01-24    460
1395-01-25    459
1395-01-26    452
1395-01-27    440
1395-01-28    433
1395-01-29    422
1395-01-30    409
1395-01-31    356
dtype: int64

In [33]:
df.columns

Index(['کد محور', 'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'سرعت متوسط', 'تعداد تخلف سرعت غیر مجاز', 'تعداد تخلف فاصله غیر مجاز',
       'تعداد تخلف سبقت غیر مجاز', 'تعداد وسیله نقلیه کلاس ۱',
       'تعداد وسیله نقلیه کلاس ۲', 'تعداد وسیله نقلیه کلاس ۳',
       'تعداد وسیله نقلیه کلاس ۴', 'تعداد وسیله نقلیه کلاس ۵', 'holiday',
       'date', 'day_of_week', 'time_start'],
      dtype='object')

In [40]:
per_code_dfs = {}
for code in df['کد محور'].unique():
    per_code_dfs[code] = df[df['کد محور']==code]

In [41]:
per_code_dfs.keys()

dict_keys([113401, 113903, 114351, 114206, 113211, 113213, 113255, 114101, 113603, 113751, 113464, 113810, 113414, 113259, 113202, 113465, 113474, 114352, 113853, 113403, 113453, 113299, 113203, 114551, 113801, 113502, 113454, 113302, 113263, 114103, 113954, 113551, 113261, 114153, 113851, 113552, 113205, 113415, 113466, 113293, 114413, 113458, 113904, 114256, 114302, 113416, 113210, 113463, 113404, 113278, 113260, 113204, 113451, 113419, 113253, 113254, 113909, 113418, 113258, 114301, 113701, 113501, 113805, 114102, 113207, 113208, 113651, 113257, 114358, 113408, 114151, 114354, 113467, 113417, 113229, 113206, 114501, 113352, 113256, 113852, 113228, 114308, 114304, 113214, 113264, 114463, 114401, 114552, 113353, 113654, 113855, 113469, 113262, 113209, 113424, 113243, 113612, 113953, 113913, 113341, 113252, 113653, 113468, 113212, 114152, 113804, 113802, 114110, 113604, 113803, 113303, 113428, 113413, 113251, 113601, 114451, 113201, 113854, 113860, 113478, 113863, 113391, 113963, 11416

In [42]:
per_code_dfs[113401].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124 entries, 0 to 30
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    124 non-null    int64  
 1   مدت زمان کارکرد (دقیقه)    124 non-null    int64  
 2   تعداد کل وسیله نقلیه       124 non-null    int64  
 3   سرعت متوسط                 124 non-null    float64
 4   تعداد تخلف سرعت غیر مجاز   124 non-null    int64  
 5   تعداد تخلف فاصله غیر مجاز  124 non-null    int64  
 6   تعداد تخلف سبقت غیر مجاز   124 non-null    int64  
 7   تعداد وسیله نقلیه کلاس ۱   124 non-null    float64
 8   تعداد وسیله نقلیه کلاس ۲   124 non-null    float64
 9   تعداد وسیله نقلیه کلاس ۳   124 non-null    float64
 10  تعداد وسیله نقلیه کلاس ۴   124 non-null    float64
 11  تعداد وسیله نقلیه کلاس ۵   124 non-null    float64
 12  holiday                    124 non-null    object 
 13  date                       124 non-null    object 


In [46]:
per_code_dfs[113401]['date'].unique()

array([jdatetime.date(1395, 1, 1), jdatetime.date(1395, 1, 2),
       jdatetime.date(1395, 1, 3), jdatetime.date(1395, 1, 4),
       jdatetime.date(1395, 1, 5), jdatetime.date(1395, 1, 6),
       jdatetime.date(1395, 1, 7), jdatetime.date(1395, 1, 8),
       jdatetime.date(1395, 1, 9), jdatetime.date(1395, 1, 10),
       jdatetime.date(1395, 1, 11), jdatetime.date(1395, 1, 12),
       jdatetime.date(1395, 1, 13), jdatetime.date(1395, 1, 14),
       jdatetime.date(1395, 1, 15), jdatetime.date(1395, 1, 16),
       jdatetime.date(1395, 1, 17), jdatetime.date(1395, 1, 18),
       jdatetime.date(1395, 1, 19), jdatetime.date(1395, 1, 20),
       jdatetime.date(1395, 1, 21), jdatetime.date(1395, 1, 22),
       jdatetime.date(1395, 1, 23), jdatetime.date(1395, 1, 24),
       jdatetime.date(1395, 1, 25), jdatetime.date(1395, 1, 26),
       jdatetime.date(1395, 1, 27), jdatetime.date(1395, 1, 28),
       jdatetime.date(1395, 1, 29), jdatetime.date(1395, 1, 30),
       jdatetime.date(1395, 1, 31)

## Other :)

In [31]:
roads = loader.get_roads()
print(list(roads.values())[:5])
print(len(roads))

['تهران - حسن آباد (جاده قديم قم)', 'حسن آباد - تهران (جاده قديم قم)', 'آبعلي - سه راهي مشاء', 'آزادراه تهران - کرج (عوارضي قديم)', 'فيروزکوه - دماوند']
133


In [ ]:
# TODO
X_train = []

In [ ]:
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# it is possible to get the prediction confidence as well
y_test_pred, y_test_pred_confidence = clf.predict(
    X_test, return_confidence=True
)  # outlier labels (0 or 1) and confidence in the range of [0,1]


In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


In [ ]:
visualize(
    clf_name,
    X_train,
    y_train,
    X_test,
    y_test,
    y_train_pred,
    y_test_pred,
    show_figure=True,
    save_figure=False,
)


In [ ]:
# save the model
dump(clf, "clf.joblib")
# load the model
clf = load("clf.joblib")
